In [1]:
import glob, os
import pandas as pd
import cv2
import numpy as np
from cv2 import __version__
print(__version__)

desfolder = "../../_data/03_tracking_result/_old_videos/maskrcnn+deepsort"
enlarged_video = "../../_data/08_historical_valid_scene/video_enlarged"

4.5.5


In [28]:
videoname = "B16_G8_Env5_0001-Scene-003"
resultfile = f'{videoname}_flipped_16X_480fps.csv'
trace = pd.read_csv(os.path.join(desfolder, resultfile)).rename(columns = {'trackID':'track_id'})
trace.sort_values('frame_id')
trace['bbox0'] = trace['x1']
trace['bbox1'] = trace['y1']
trace['bbox2'] = trace['x2']-trace['x1']
trace['bbox3'] = trace['y2']-trace['y1']
trace.head()

,frame_id,track_id,class,x1,y1,x2,y2,loc_x,loc_y,bbox0,bbox1,bbox2,bbox3
0,31,2,person,302,236,702,433,502.0,433,302,236,400,197
1,31,2,person,302,236,702,433,502.0,433,302,236,400,197
2,31,2,person,302,236,702,433,502.0,433,302,236,400,197
3,31,2,person,302,236,702,433,502.0,433,302,236,400,197
4,32,2,person,323,237,711,441,517.0,441,323,237,388,204


In [47]:
# exclude the extreme large bbox2 and bbox3
trace = trace[trace['bbox2']<=40].reset_index(drop = True)

trace = trace[trace['bbox3']<=100].reset_index(drop = True)

In [29]:
# visualize the tracking result on the video
# resultfile = 'B16_G8_Env5_0001-Scene-003_flipped.txt'
# trace = pd.read_csv(os.path.join(desfolder, resultfile), sep=',', header=None)
# trace.columns = ['frame_id','track_id', 'bbox0', 'bbox1', 'bbox2', 'bbox3', 'class_id', -1, -1, -1]
# for x in ["bbox0", "bbox1", "bbox2", "bbox3"]:
#     trace[x] = trace[x].astype(int)

In [48]:
def getbasics(file_path):
    video = cv2.VideoCapture(file_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    print('frames per second =',fps)
    size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    print('frames size =',size)
    # video.release()
    return video, fps, size

palette = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)


def compute_color_for_labels(label):
    """
    Simple function that adds fixed color depending on the class
    """
    color = [int((p * (label ** 2 - label + 1)) % 255) for p in palette]
    return tuple(color)

In [49]:
videofolder = "../../_data/08_historical_valid_scene/video_enlarged"

file_path = os.path.join(videofolder, "B16_G8_Env5_0001-Scene-003_flipped_16X_480fps.mp4")
video, fps, size = getbasics(file_path)
startframe  = 31
video.set(cv2.CAP_PROP_POS_FRAMES, startframe)

frames per second = 479.52
frames size = (720, 480)


True

In [50]:

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
video_out_name = videoname+ "viz"

vizfolder = "../../_data/03_tracking_result/_old_videos_viz"
video_out = cv2.VideoWriter(os.path.join(vizfolder, 
                                         f"{video_out_name}.mp4"), fourcc, fps, size)



In [51]:
ret = True
count = startframe

while ret and count<startframe+3000:
    ret, frame = video.read()
    frame_id = count
    data = trace[trace["frame_id"]==frame_id].reset_index(drop = True)
    # only viz the stationary people
    # data = temp[temp['stationary_1']==1].reset_index(drop = True)
    
    if data.shape[0]>0:
        for i in range(data.shape[0]):
            track_id = data.at[i,'track_id']
            color = compute_color_for_labels(track_id)
            # cv2.rectangle(frame, 
            #             (data.at[i,'bbox0'], 
            #             data.at[i,'bbox1']), 
            #             (data.at[i,'bbox0']+data.at[i,'bbox2'], 
            #             data.at[i,'bbox1']+data.at[i,'bbox3']), 
            #                 color, 2)
            cv2.rectangle(frame,
                         (data.at[i,'x1'], 
                        data.at[i,'y1']), 
                        (data.at[i,'x2'], 
                        data.at[i,'y2']), 
                            color, 2)

        
    else:
        print("no observation at frame", frame_id)
    video_out.write(frame) 
    count = count+1
    # print(ret)
video_out.release()

no observation at frame 31
no observation at frame 32
no observation at frame 63
no observation at frame 64
no observation at frame 65
no observation at frame 66
no observation at frame 67
no observation at frame 93
no observation at frame 94
no observation at frame 95
no observation at frame 96
no observation at frame 97
no observation at frame 98
no observation at frame 99
no observation at frame 100
no observation at frame 537
no observation at frame 539
no observation at frame 540
no observation at frame 541
no observation at frame 572
no observation at frame 573
no observation at frame 574
no observation at frame 603
no observation at frame 604
no observation at frame 607
no observation at frame 608
no observation at frame 609
no observation at frame 611
no observation at frame 612
no observation at frame 613
no observation at frame 614
no observation at frame 615
no observation at frame 616
no observation at frame 704
no observation at frame 705
no observation at frame 815
no obs